<a href="https://colab.research.google.com/github/Sandy200531/image_classification/blob/main/CNN_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simple DL architecture for image classification**
## **It consists 3 convolution and 1 fully connected layers**
We are using Cats vs Dogs Dataset as it is binary classification task




In [1]:
!git clone https://github.com/Sandy200531/image-calassification.git

Cloning into 'image-calassification'...


In [ ]:
# Import required libs and dataloading

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd

In [ ]:
# Load and preprocess data
# Load the Cats vs Dogs dataset from TensorFlow Datasets
data, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

# Split the dataset into training, validation, and test sets
train_data = data['train'].take(17500)  # 70% of the dataset
val_data = data['train'].skip(17500).take(3750)  # 15% of the dataset
test_data = data['train'].skip(21250)  # 15% of the dataset


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteNQZVDO/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
# Load the dataset
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Define the batch size and image size
BATCH_SIZE = 32
IMG_SIZE = 224

# Preprocess the data
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.cast(label, tf.int32)
    return image, label

ds_train = ds_train.map(preprocess).batch(BATCH_SIZE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE)


**Define the model architecture**

In [ ]:
tf.keras.backend.clear_session()
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

**Train and Test the model**

In [ ]:
# Train the model
history = model.fit(ds_train,
                    validation_data=ds_test,
                    epochs=10)

# Evaluate the model
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10
582/582 [==============================] - 2109s 4s/step - loss: 0.6377 - accuracy: 0.6255 - val_loss: 0.5277 - val_accuracy: 0.7343
Epoch 2/10
582/582 [==============================] - 2078s 4s/step - loss: 0.5052 - accuracy: 0.7500 - val_loss: 0.4738 - val_accuracy: 0.7739
Epoch 3/10
582/582 [==============================] - 2124s 4s/step - loss: 0.4214 - accuracy: 0.8036 - val_loss: 0.4569 - val_accuracy: 0.7799
Epoch 4/10
582/582 [==============================] - 2029s 3s/step - loss: 0.3201 - accuracy: 0.8586 - val_loss: 0.4820 - val_accuracy: 0.8042
Epoch 5/10
582/582 [==============================] - 2055s 4s/step - loss: 0.2072 - accuracy: 0.9141 - val_loss: 0.5322 - val_accuracy: 0.8029
Epoch 6/10
582/582 [==============================] - 2040s 4s/step - loss: 0.1360 - accuracy: 0.9466 - val_loss: 0.7995 - val_accuracy: 0.7857
Epoch 7/10
582/582 [==============================] - 2028s 3s/step - loss: 0.0994 - accuracy: 0.9643 - val_loss: 0.7501 - val_accuracy: